In [2]:
#from utils import process_matchms as _myfun
import plotly.express as px
import pickle
import kmedoids
import numpy as np
from sklearn import manifold
import itertools
from plotly.colors import n_colors
import pandas as pd


In [ ]:

# Load similarity matrices.
file = open("data/sim_matrix_ms2deepscore", 'rb')
sm = pickle.load(file) # sm = _myfun.extract_similarity_scores(sm) # <- if cosine based
file.close()
dist = 1.-sm
dist = np.round(dist, 5)

# Process cosine

modified cosine sm matrix is actual a matrix of tuples with frag overlap and sim score.

In [ ]:
file = open("data/sim_matrix_cosine_modified", 'rb')
sm = pickle.load(file) 
sm = _myfun.extract_similarity_scores(sm)
file.close()

with open('data/sim_matrix_cosine_modified_extracted.pickle', 'wb') as handle:
    pickle.dump(sm, handle)

# Construct Class Table

In [3]:
# Load Pandas Classification Table & Extract classes
file = open("data/pandas-classification-table-0012.pickle", 'rb')
class_table = pickle.load(file)
file.close()


In [4]:
class_table

,idx,inchi_key,smiles,cf_kingdom,cf_superclass,cf_class,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,npc_isglycoside
0,0,XTJNPXYDTGJZSA-PKOBYXMFSA-N,OC(=O)[C@H](NC(=O)CCN1C(=O)[C@@H]2Cc3ccccc3CN2...,Organic compounds,Organic acids and derivatives,Peptidomimetics,Hybrid peptides,Hybrid peptides,NaN,NaN,Amino acids and Peptides,0
1,1,VOYWJNWCKFCMPN-FHERZECASA-N,O=C(N1[C@@H](C#N)C2OC2c3ccccc13)c4ccccc4,Organic compounds,Organoheterocyclic compounds,Quinolines and derivatives,Hydroquinolines,Hydroquinolines,Chalcones,Flavonoids,Shikimates and Phenylpropanoids,0
2,2,IRZVHDLBAYNPCT-UHFFFAOYSA-N,COc1cc(O)c2c(=O)cc(oc2c1)c3ccccc3,Organic compounds,Phenylpropanoids and polyketides,Flavonoids,O-methylated flavonoids,7-O-methylated flavonoids,Flavones,Flavonoids,Shikimates and Phenylpropanoids,0
3,3,OPWCHZIQXUKNMP-RGEXLXHISA-N,COc1c2OCOc2cc(CCN(C)C(=O)c3ccccc3)c1/C=C\4/C(=...,Organic compounds,Organoheterocyclic compounds,Benzodioxoles,NaN,Benzodioxoles,NaN,NaN,Alkaloids,0
4,4,GTBYYVAKXYVRHX-BVSLBCMMSA-N,CC(C)[C@H](NC(=O)N1[C@@H](C(C)C)C(=O)Nc2ccccc1...,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Dipeptides,Dipeptides,Small peptides,Amino acids and Peptides,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1262,1262,KQYNNEFXSOOTEY-OGLOGDKOSA-N,CC(NC(=O)[C@H](Cc1c[nH]c2ccccc12)NC(=O)[C@@H]3...,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Dipeptides,Tripeptides,Small peptides,Amino acids and Peptides,0
1263,1263,QNTBHYIWBSPSMN-SHTZXODSSA-N,Cc1c(OCC(=O)NC[C@H]2CC[C@@H](CC2)C(=O)O)ccc3c4...,Organic compounds,Phenylpropanoids and polyketides,Coumarins and derivatives,NaN,Coumarins and derivatives,Simple coumarins,Coumarins,Shikimates and Phenylpropanoids,0
1264,1264,LWRMZXDWYOOEPB-UHFFFAOYSA-N,COC(=O)c1cc(NC(=O)CN2CCc3cc4OCCCOc4cc3C2)cc(c1...,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzoic acids and derivatives,Acylaminobenzoic acid and derivatives,Isoquinoline alkaloids,Tyrosine alkaloids,Alkaloids,0
1265,1265,BAKYVUHOODEWGV-UHFFFAOYSA-N,COC(=O)C1=COC(OC2OC(CO)C(O)C(O)C2O)C(C=C)C1CC3...,Organic compounds,Lipids and lipid-like molecules,Prenol lipids,Terpene glycosides,Terpene glycosides,Secoiridoid monoterpenoids,Monoterpenoids,Terpenoids,1


In [5]:
#print(class_table.shape)
#print(class_table.iloc[0])
class_table = class_table.replace(" ","_",regex=True )
print(class_table.iloc[0])
for k in [10, 20, 25, 30, 35, 40, 45, 50, 100]:
    # kmedoid clustering
    cluster = kmedoids.KMedoids(n_clusters=k, metric='precomputed', random_state=0, method = "fasterpam")  
    cluster_km = cluster.fit_predict(dist)
    cluster_km = ["KM_" + str(elem) for elem in cluster_km]
    class_table[f"kmedoid_{k}"] = cluster_km

class_table.head()
print(list(class_table.columns))
# class_table = class_table.drop("npc_isglycoside", axis = 1)
# class_table = class_table.iloc[:, 3:]

print(list(class_table.columns))


idx                                                                       0
inchi_key                                       XTJNPXYDTGJZSA-PKOBYXMFSA-N
smiles                    OC(=O)[C@H](NC(=O)CCN1C(=O)[C@@H]2Cc3ccccc3CN2...
cf_kingdom                                                Organic_compounds
cf_superclass                                 Organic_acids_and_derivatives
cf_class                                                    Peptidomimetics
cf_subclass                                                 Hybrid_peptides
cf_direct_parent                                            Hybrid_peptides
npc_class_results                                                       NaN
npc_superclass_results                                                  NaN
npc_pathway_results                                Amino_acids_and_Peptides
npc_isglycoside                                                           0
Name: 0, dtype: object


NameError: name 'dist' is not defined

In [ ]:

class_table.to_csv('data/classification_table.csv', encoding='utf-8', index = False)

# Generate TSNE Embedding

In [ ]:
# Compute T-sne embedding
model = manifold.TSNE(metric="precomputed", random_state = 123, perplexity= 15)
z = model.fit_transform(dist)
x_coords = z[:,0] * 100 # scale the coordinate space for better visualization
y_coords = z[:,1] * 100 # scale the coordinate space for better visualization
id = np.arange(0, len(x_coords), 1)
tsne_df = pd.DataFrame({"x": x_coords, "y":y_coords, "id":id})

with open('data/tsne_df.pickle', 'wb') as handle:
    pickle.dump(tsne_df, handle)

# Generate precursor mz values vector

In [11]:
file = open("data/cleaned_demo_data.pickle", 'rb')
spectra = pickle.load(file) 
file.close()
spectrum = spectra[0]

mz = [round(spectrum.get("precursor_mz"), 2) for spectrum in spectra]
mz = np.array(mz)

with open('data/extracted_precursor_mz_values.pickle', 'wb') as handle:
    pickle.dump(mz, handle)